# Typical use cases

## Problem

In this notebook, we provide examples of different solver configurations a for a sample Knapsack Problem with three items. The goal is to put the selected items in the knapsack to achieve the maximal value with total weight not exceeding the `max_weight`.  The `item_weights` and `item_values` fields specify the weight and value of each item, respectively.

```yaml
problem:
  type: KnapsackProblem
  max_weight: 2
  item_weights: [1, 1, 1]
  item_values: [2, 2, 1]
```

## Configuration for the QAOA
The configuration below shows how to create a QAOA instance with 5 layers and a local gradient descent optimizer (`QmlGradientDescent`) --- by default the `adam` optimizer.

The `gamma` and `beta` keys indicate variational parameters searched by specified optimizer, while the `penalty_weights` refer to the initial penalty weights in the objective function of the Knapsack Problem.

In [1]:
qaoa_config_yaml = """
problem:
    type: KnapsackProblem
    max_weight: 2
    item_weights: [1, 1, 1]
    item_values: [2, 2, 1]
solver:
    category: gate_based
    platform: pennylane
    name: QAOA
    layers: 5
    gamma:
        init: [0.5, 0.5, 0.5, 0.5, 0.5]
    beta:
        init: [1., 1., 1., 1., 1.]
    optimizer: 
        type: QmlGradientDescent
    penalty_weights: [1, 2.5, 2.5]
"""

In [2]:
import yaml
from QHyper.solvers import solver_from_config

qaoa_config = yaml.safe_load(qaoa_config_yaml)
solver = solver_from_config(qaoa_config)

In [3]:
results = solver.solve()
results.probabilities

rec.array([(0, 0, 0, 0, 0, 0.01925662), (0, 0, 0, 0, 1, 0.00499353),
           (0, 0, 0, 1, 0, 0.06602495), (0, 0, 0, 1, 1, 0.00135034),
           (0, 0, 1, 0, 0, 0.00929764), (0, 0, 1, 0, 1, 0.00077379),
           (0, 0, 1, 1, 0, 0.0150905 ), (0, 0, 1, 1, 1, 0.00105546),
           (0, 1, 0, 0, 0, 0.01907666), (0, 1, 0, 0, 1, 0.0119462 ),
           (0, 1, 0, 1, 0, 0.01663426), (0, 1, 0, 1, 1, 0.0009503 ),
           (0, 1, 1, 0, 0, 0.0058822 ), (0, 1, 1, 0, 1, 0.11219791),
           (0, 1, 1, 1, 0, 0.01533116), (0, 1, 1, 1, 1, 0.00056568),
           (1, 0, 0, 0, 0, 0.01907666), (1, 0, 0, 0, 1, 0.0119462 ),
           (1, 0, 0, 1, 0, 0.01663426), (1, 0, 0, 1, 1, 0.0009503 ),
           (1, 0, 1, 0, 0, 0.0058822 ), (1, 0, 1, 0, 1, 0.11219791),
           (1, 0, 1, 1, 0, 0.01533116), (1, 0, 1, 1, 1, 0.00056568),
           (1, 1, 0, 0, 0, 0.00744606), (1, 1, 0, 0, 1, 0.07094647),
           (1, 1, 0, 1, 0, 0.01561782), (1, 1, 0, 1, 1, 0.0122494 ),
           (1, 1, 1, 0, 0, 0.00071

## Configuration for the D-Wave Advantage solver

Configuration for a grid search hyperoptimizer. The objective function penalties (`hyper_args`) are searched within specified `bounds` using provided `steps`. The objective function is solved with a quantum annealing device --- D-Wave Advantage.

<div class="alert alert-warning">
In order to run the code below, you need to have access to the D-Wave Advantage device. Obtain a token at <a href="https://cloud.dwavesys.com/leap/">D-Wave Leap</a>.
</div>

In [4]:
advantage_config_yaml = """
problem:
    type: knapsack
    max_weight: 2
    item_weights: [1, 1, 1]
    item_values: [2, 2, 1]
solver:
    category: quantum_annealing
    platform: d_wave
    name: Advantage
    num_reads: 100
hyper_optimizer:
    optimizer: 
        type: grid
    penalty_weights: 
        min: [1, 1, 1]
        max: [3, 3, 3]
        step: [1, 1, 1]
"""

In [5]:
import yaml
from QHyper.solvers import solver_from_config

config = yaml.safe_load(advantage_config_yaml)

hyper_optimizer = solver_from_config(config)

In [6]:
hyper_optimizer.solve()
results = hyper_optimizer.run_with_best_params()

results.probabilities

rec.array([(1, 1, 0, 0, 1, 0.52, -4.), (1, 0, 1, 0, 1, 0.04, -3.),
           (1, 1, 1, 0, 1, 0.1 , -3.), (1, 1, 1, 1, 1, 0.13, -3.),
           (0, 1, 1, 0, 1, 0.1 , -3.), (1, 1, 0, 1, 0, 0.01, -2.),
           (1, 0, 0, 1, 0, 0.02, -2.), (0, 1, 0, 1, 0, 0.08, -2.)],
          dtype=[('x0', '<i8'), ('x1', '<i8'), ('x2', '<i8'), ('x3', '<i8'), ('x4', '<i8'), ('probability', '<f8'), ('energy', '<f8')])

## Advanced configuration

QHyper configuration of the QAOA variant (WF-QAOA) with 5 `layers` and  the local gradient descent Adam `optimizer` (qml). `angles` indicate  initial variational parameters optimized by the method. `hyper_args` refer to the initial objective function penalties searched within `hyper_optimizer` `bounds` by the `CEM` method. `processes`, `samples_per_epoch`, and `epochs` are parameters specific to the `CEM` method.

In [7]:
advanced_config_yaml = """
problem:
    type: KnapsackProblem
    max_weight: 2
    item_weights: [1, 1, 1]
    item_values: [2, 2, 1]
solver:
    category: gate_based
    platform: pennylane
    name: QAOA
    layers: 5
    gamma:
        init: [0.5, 0.5, 0.5, 0.5, 0.5]
    beta:
        init: [1., 1., 1., 1., 1.]
    optimizer: 
        type: QmlGradientDescent
        name: adam
        steps: 10
        stepsize: 0.05
hyper_optimizer:
    optimizer:
        type: cem
        processes: 4
        samples_per_epoch: 200
        epochs: 5
        disable_tqdm: False
    penalty_weights:
        min: [1, 1, 1]
        max: [10, 10, 10]
        init: [1, 2.5, 2.5]
"""

In [8]:
import yaml
from QHyper.solvers import solver_from_config

config = yaml.safe_load(advanced_config_yaml)

hyper_optimizer = solver_from_config(config)

In [9]:
hyper_optimizer.solve()

100%|██████████| 200/200 [00:29<00:00,  6.68it/s]


OptimizationResult(value=-2.115995908832305, params=array([1.33150937, 3.06774974, 2.27513882]), history=[[OptimizationResult(value=-2.1040190805897265, params=array([1.05348285, 2.76970457, 2.23129254]), history=[[OptimizationResult(value=-2.1040190805897265, params=array([1.05348285, 2.76970457, 2.23129254]), history=[])]]), OptimizationResult(value=-0.2560912669193908, params=array([2.34757359, 2.59879284, 4.11176842]), history=[[OptimizationResult(value=-0.2560912669193908, params=array([2.34757359, 2.59879284, 4.11176842]), history=[])]]), OptimizationResult(value=-0.5692961783152712, params=array([2.64453033, 4.55754873, 1.99116314]), history=[[OptimizationResult(value=-0.5692961783152712, params=array([2.64453033, 4.55754873, 1.99116314]), history=[])]]), OptimizationResult(value=-1.5314189012300496, params=array([1.72882425, 2.97751889, 2.31197551]), history=[[OptimizationResult(value=-1.5314189012300496, params=array([1.72882425, 2.97751889, 2.31197551]), history=[])]]), Optim

In [10]:
results = hyper_optimizer.run_with_best_params()

## Results evaluation

After obtaining the results, we evaluate the solution by calculating the total cost and weight of the items in the knapsack.

In [11]:
from QHyper.util import (
    weighted_avg_evaluation, sort_solver_results, add_evaluation_to_results)

problem = solver.problem

# Evaluate results with weighted average evaluation
print("Evaluation:")
print(weighted_avg_evaluation(
    results.probabilities, problem.get_score,
    penalty=0, limit_results=10, normalize=True
))
print("Sorted results:")
sorted_results = sort_solver_results(
    results.probabilities, limit_results=5)
print(sorted_results)

# Add evaluation to results
results_with_evaluation = add_evaluation_to_results(
    sorted_results, problem.get_score, penalty=0)

for rec in results_with_evaluation:
    print(f"Result: {rec}, "
          f"Prob: {rec['probability']:.5}, "
          f"Evaluation: {rec['evaluation']:.5}")

Evaluation:
-2.539611243042654
Sorted results:
[(1, 1, 0, 0, 1, 0.2256414 ) (1, 0, 1, 0, 1, 0.18556845)
 (0, 1, 1, 0, 1, 0.18556845) (1, 1, 1, 0, 0, 0.03122013)
 (1, 0, 0, 0, 0, 0.03119786)]
Result: (1, 1, 0, 0, 1, 0.2256414, -4.), Prob: 0.22564, Evaluation: -4.0
Result: (1, 0, 1, 0, 1, 0.18556845, -3.), Prob: 0.18557, Evaluation: -3.0
Result: (0, 1, 1, 0, 1, 0.18556845, -3.), Prob: 0.18557, Evaluation: -3.0
Result: (1, 1, 1, 0, 0, 0.03122013, 0.), Prob: 0.03122, Evaluation: 0.0
Result: (1, 0, 0, 0, 0, 0.03119786, 0.), Prob: 0.031198, Evaluation: 0.0
